#  [**Just Relax It**](https://github.com/intsystems/discrete-variables-relaxation) Demo Сode

In this notebook, we demonstrate how the methods from the **Just Relax It** library work.

> Before you start experimenting with the library, you need to install all the necessary dependencies, follow the instruction in the `README.md`.

In the **Just Relax It** library, the implemented distributions have a similar structure, so we chose one distribution for demonstration and conducted a number of experiments with it. [**Correlated Relaxed Bernoulli**](https://openreview.net/pdf?id=oDFvtxzPOx) was chosen as a demonstration method.
This method generates correlated gate vectors from a multivariate Bernoulli distribution using a Gaussian copula:
$$C_R(U_1, \ldots, U_p) = \Phi_R(\Phi^{-1}(U_1), \ldots, \Phi^{-1}(U_p)),
$$
where $\Phi_R$ is the joint CDF of a multivariate Gaussian distribution with correlation matrix $R$, and $\Phi^{-1}$ is the inverse CDF of the standard univariate Gaussian distribution.
The gate vector $m$ is generated as:
$$m_k =
\begin{cases}
1 & \text{if } U_k \leq \pi_k \\0 & \text{if } U_k > \pi_k
\end{cases}\quad k = 1, \ldots, p,
$$
where $U_k$ are correlated random variables preserving the input feature correlations.
For differentiability, a continuous relaxation is applied:
$$m_k = \sigma \left( \frac{1}{\tau} \left( \log \frac{U_k}{1 - U_k} + \log \frac{\pi_k}{1 - \pi_k} \right) \right),
$$
where $\sigma(x) = \frac{1}{1 + \exp(-x)}$ is the sigmoid function, and $\tau$ is a temperature hyperparameter.

The theory of other methods from **Just Relax It** library can be viewed in the [blog-post](https://github.com/intsystems/discrete-variables-relaxation/blob/main/assets/blog-post.pdf).

## Importing libraries

In [3]:
import pyro.distributions as dist
from PIL import Image
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, "..", "src")))
from relaxit.distributions import CorrelatedRelaxedBernoulli

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simplest Examples
### Initializing distribution **Correlated Relaxed Bernoulli**

Let's define the parameters $\pi$, $R$, and $\tau$ for the demonstration:

- Tensor $\pi$, representing the probabilities of the Bernoulli distribution, with an event shape of 3 and a batch size of 2.
   $$
   \pi = \begin{bmatrix}
   0.2 & 0.4 & 0.4 \\
   0.3 & 0.5 & 0.2
   \end{bmatrix}
   $$

- Correlation matrix $R$ for the Gaussian copula.
   $$
   R = \begin{bmatrix}
   1.0 & 0.5 & 0.3 \\
   0.5 & 1.0 & 0.7 \\
   0.3 & 0.7 & 1.0
   \end{bmatrix}
   $$

- Temperature hyperparameter $\tau$.
   $$
   \tau = 0.1
   $$

In [20]:
torch.manual_seed(0)

# Define the parameters
pi = torch.tensor([[0.2, 0.4, 0.4], [0.3, 0.5, 0.2]])
R = torch.tensor([[1.0, 0.5, 0.3], [0.5, 1.0, 0.7], [0.3, 0.7, 1.0]])
tau = torch.tensor(0.1)

# Create an instance of the CorrelatedRelaxedBernoulli distribution
MyCorrelatedRelaxedBernoulli = CorrelatedRelaxedBernoulli(pi=pi, R=R, tau=tau)

# Example usage
sample = MyCorrelatedRelaxedBernoulli.rsample()
print("Sampled gate vector:\n", sample)

Sampled gate vector:
 tensor([[1.0000e+00, 9.8642e-01, 5.6586e-12],
        [6.7509e-01, 2.3547e-05, 2.5322e-18]])
